In [ ]:
%pip install ccxt==4.4.71 pandas==2.2.2

In [ ]:
import ccxt
import pandas as pd
from datetime import datetime, timezone, timedelta

In [ ]:
# @title Введите id биржи https://docs.ccxt.com/#/?id=exchanges
exchange_id = "binanceus" # @param {"type":"string"}
exchange = getattr(ccxt, exchange_id)()
exchange.enableRateLimit = True

In [ ]:
# @title Введите https proxy для получения данных с Binance или Bybit
https_proxy = "" # @param {"type":"string","placeholder":"Leave blank if you do not want to use a proxy"}
exchange.httpsProxy = https_proxy

In [ ]:
# @title Выберите дату c которой скачивать данные
start_date = "2025-03-01" # @param {"type":"date"}

In [ ]:
# @title Выберите дату до которой скачивать данные
end_date = "2025-03-31" # @param {"type":"date"}

In [ ]:
exchange.load_markets()
for symbol in exchange.markets:
    if symbol.endswith('/USDT'):
        print(symbol)

BTC/USDT
ETH/USDT
XRP/USDT
BCH/USDT
LTC/USDT
BNB/USDT
ADA/USDT
BAT/USDT
ETC/USDT
XLM/USDT
ZRX/USDT
DOGE/USDT
ATOM/USDT
NEO/USDT
VET/USDT
QTUM/USDT
ONT/USDT
KNC/USDT
VTHO/USDT
COMP/USDT
HNT/USDT
MKR/USDT
ONE/USDT
BAND/USDT
STORJ/USDT
BUSD/USDT
UNI/USDT
SOL/USDT
EGLD/USDT
PAXG/USDT
OXT/USDT
ZEN/USDT
FIL/USDT
AAVE/USDT
GRT/USDT
SHIB/USDT
CRV/USDT
AXS/USDT
AVAX/USDT
CTSI/USDT
DOT/USDT
YFI/USDT
1INCH/USDT
FTM/USDT
USDC/USDT
MATIC/USDT
MANA/USDT
ALGO/USDT
LINK/USDT
EOS/USDT
ZEC/USDT
ENJ/USDT
NEAR/USDT
OMG/USDT
SUSHI/USDT
LRC/USDT
LPT/USDT
POLY/USDT
NMR/USDT
SLP/USDT
ANT/USDT
CHZ/USDT
OGN/USDT
GALA/USDT
TLM/USDT
SNX/USDT
AUDIO/USDT
ENS/USDT
REQ/USDT
APE/USDT
FLUX/USDT
TRX/USDT
COTI/USDT
VOXEL/USDT
RLC/USDT
UST/USDT
BICO/USDT
API3/USDT
BNT/USDT
IMX/USDT
SPELL/USDT
JASMY/USDT
FLOW/USDT
GTC/USDT
THETA/USDT
TFUEL/USDT
OCEAN/USDT
LAZIO/USDT
SANTOS/USDT
ALPINE/USDT
PORTO/USDT
REN/USDT
CELR/USDT
SKL/USDT
VITE/USDT
WAXP/USDT
LTO/USDT
FET/USDT
BOND/USDT
LOKA/USDT
ICP/USDT
T/USDT
OP/USDT
ROSE/USDT
CELO

In [ ]:
def get_ohlcv(symbol, limit, timeframe, exchange, start_date, end_date):
    all_ohlcv = []
    start_dt = datetime.strptime(start_date, '%Y-%m-%d').replace(tzinfo=timezone.utc)
    end_dt = datetime.strptime(end_date, '%Y-%m-%d').replace(hour=23, minute=59, second=59, microsecond=999999, tzinfo=timezone.utc)
    since_timestamp = int(start_dt.timestamp() * 1000)
    end_timestamp = int(end_dt.timestamp() * 1000)
    timeframe_duration_ms = exchange.parse_timeframe(timeframe) * 1000
    try:
        while since_timestamp < end_timestamp:
          ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since_timestamp, limit=limit)
          if not ohlcv:
              break
          all_ohlcv.extend(ohlcv)
          since_timestamp = ohlcv[-1][0] + timeframe_duration_ms

        symbol_data = pd.DataFrame(all_ohlcv, columns=['Date', 'O', 'H', 'L', 'C', 'V'])
        symbol_data.set_index('Date', inplace=True)
        symbol_data.index = pd.to_datetime(symbol_data.index, unit='ms', utc=True)
        symbol_data = symbol_data[~symbol_data.index.duplicated(keep='first')]
        symbol_data = symbol_data[(symbol_data.index >= start_dt) & (symbol_data.index <= end_dt)]
        return symbol_data
    except Exception as e:
        print('Trouble with downloading data\n')
        print(e)

In [ ]:
df = get_ohlcv(symbol = 'BTC/USDT', limit=1000, timeframe='1m', exchange=exchange, start_date=start_date, end_date=end_date)

In [ ]:
df

,O,H,L,C,V
Date,,,,,
2025-03-01 00:00:00+00:00,84573.84,84573.84,84572.07,84572.07,0.04735
2025-03-01 00:01:00+00:00,84254.59,84254.59,84254.59,84254.59,0.00103
2025-03-01 00:02:00+00:00,84254.59,84254.59,84254.59,84254.59,0.00000
2025-03-01 00:03:00+00:00,84254.59,84254.59,84254.59,84254.59,0.00000
2025-03-01 00:04:00+00:00,84254.59,84254.59,84254.59,84254.59,0.00000
...,...,...,...,...,...
2025-03-30 23:55:00+00:00,82384.92,82384.92,82384.92,82384.92,0.00000
2025-03-30 23:56:00+00:00,82384.92,82384.92,82384.92,82384.92,0.00000
2025-03-30 23:57:00+00:00,82406.17,82406.17,82406.17,82406.17,0.00002


In [ ]:
df.to_csv('BTCUSDT_1m.csv')